# **Image Classification Using SVD**  
### **Python Code for Image Classification Using Singular Value Decomposition and Optimization**

#### Authors: * yomna abdelmegeed , nadia Ashraf , monica maged , yara ahmed , hagar atef , menna alla ahmed*

---

### Overview  
This repository contains Python code accompanying our paper:  
[**Image Classification Using Singular Value Decomposition and Optimization**](https://arxiv.org/pdf/2412.07288).

This code demonstrates the implementation of our proposed method for image classification using singular value decomposition (SVD) and optimized categorical representative images.  

---

## Outline of the Code:


1.   Import packages & connect to drive
2.   Image pre-processing
3.   Split data into training and testing
4.   Compute templates with training set
5.   Classification probability distribution and errors with different ranks and norms, testing against optimally weighted template
6.   Norm evaluation at a best rank
7.   Original test images with predicted labels for Fro norm rank 10
8.   Generate subplot showing best rank images for one persian cat and boxer dog test for each norm
9.   Further comparison of optimally weighted template vs average template
10.  Single image experiments



## 1. Import Packages 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from skimage.exposure import equalize_hist
from skimage import io, color
from skimage.transform import rotate
from skimage.transform import resize
from skimage.transform import rescale
from skimage.exposure import adjust_gamma
from sklearn.model_selection import StratifiedShuffleSplit
from numpy.linalg import svd
from scipy.optimize import minimize
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

## 2. Image Pre-Processing
### Resize and Convert to Grayscale

In [3]:
# Define paths to the folders containing images
boxer_folder = 'boxer'
persian_cat_folder = 'persian_cat'

#image_size = (256, 256)  # Resize all images to this size
image_size = (64, 64) 

# Function to load and preprocess images from a folder
def load_images(folder_path, image_size):
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith(('.jpg', '.JPG', '.jpeg')):
            img = io.imread(os.path.join(folder_path, filename))

            # Ensure the image is RGB (3 channels) by removing an alpha channel if it exists
            if img.shape[-1] == 4:
                img = img[..., :3]  # Keep only the first 3 channels (RGB)

            img_gray = color.rgb2gray(img)  # Convert to grayscale
            img_resized = resize(img_gray, image_size)  # Resize
            images.append(img_resized)  # Store resized image
    return np.array(images)  # converts the list of images to a NumPy array

In [4]:
# Function to augment images with rotations/flips and zooms (*4)
def augment_images(images):
    augmented = []
    for img in images:
        augmented.append(img)
        augmented.append(rotate(img, angle=10))
        augmented.append(np.fliplr(img))
        #  zoom by 30%
        zoomed = rescale(img, 1.3, mode='reflect', anti_aliasing=True)
        # crop zoomed image
        center = zoomed.shape[0] // 2
        cropped = zoomed[center - 32:center + 32, center - 32:center + 32]
        augmented.append(cropped)
    return np.array(augmented)
    

# Load images
boxer_images = load_images(boxer_folder, image_size)
persian_cat_images = load_images(persian_cat_folder, image_size)

## 3. Split data into training and testing